In [1]:
import cv2, os, pyautogui, time
import matplotlib.pyplot as plt
from src.tools import *

width, height= pyautogui.size()

def draw_bound(bound, frame):
    x_max, x_min, y_max, y_min = bound
    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2)

In [6]:
cap = cv2.VideoCapture(0)

try:
    while(True): 
        ret, frame = cap.read()
        try:
            preds = landmarks_detector.get_landmarks(frame)
        except:
            pass
        try:
            left_eye = get_bound(preds[36:42])
            right_eye = get_bound(preds[42:48])
            middle_list = [21, 22, 28, 39 ,42]
            middle = get_bound(preds[middle_list])
            draw_bound(left_eye, frame)
            draw_bound(right_eye, frame)
            draw_bound(middle, frame)
        except:
            pass
        cv2.imshow('preview',frame)
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
#         X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [4]:
left_eye = get_bound(preds[36:42])
right_eye = get_bound(preds[42:48])
middle_list = [21, 22, 28, 39 ,42]
middle = get_bound(preds[middle_list])

# Collect

In [31]:
X, X_temp, Y = [], [], []

In [32]:
cap = cv2.VideoCapture(0)
frame_width, frame_height = 320, 240
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass

target_height, target_width = 32, 64

x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(10):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 2000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            if ret:
                X_temp.append(frame)
                Y.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [33]:
from ipywidgets import IntProgress
from IPython.display import display

target_height, target_width = 32, 64
max_count = len(X_temp)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

X = []
for frame in X_temp:
    preds = landmarks_detector.get_landmarks(frame)
    try:
        left_eye = get_bound(preds[36:42])
        box = get_box(left_eye)
    except:
        pass
    X.append(warpBox(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), box, target_height=target_height, target_width=target_width))
    f.value += 1

IntProgress(value=0, max=2100)

In [34]:
X = np.array(X)/255.0
X = np.expand_dims(X, -1)
Y = np.array(Y)

print(X.shape, Y.shape)

(2100, 32, 64, 1) (2100, 2)


In [35]:
X = np.concatenate([X, X_old])
Y = np.concatenate([Y, Y_old])

In [24]:
X_old, Y_old = X, Y

Model training

In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_height, target_width, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 62, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 14, 64)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 12, 128)        73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 6, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1536)             

In [41]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(X))), train_size=0.9, shuffle=True)

In [43]:
history = model.fit(X[train_set], Y[train_set], batch_size=256, epochs=100, validation_data=(X[test_set], Y[test_set]))

Epoch 1/100
15/15 [==============================] - 2s 158ms/step - loss: 0.0021 - val_loss: 0.0024
Epoch 2/100
15/15 [==============================] - 2s 158ms/step - loss: 0.0023 - val_loss: 0.0028
Epoch 3/100
15/15 [==============================] - 2s 164ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 4/100
15/15 [==============================] - 3s 175ms/step - loss: 0.0019 - val_loss: 0.0022
Epoch 5/100
15/15 [==============================] - 3s 178ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 6/100
15/15 [==============================] - 3s 173ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 7/100
15/15 [==============================] - 3s 177ms/step - loss: 0.0018 - val_loss: 0.0024
Epoch 8/100
15/15 [==============================] - 3s 176ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 9/100
15/15 [==============================] - 3s 175ms/step - loss: 0.0023 - val_loss: 0.0033
Epoch 10/100
15/15 [==============================] - 3s 174ms/step - loss: 0.0024 - val_lo

15/15 [==============================] - 3s 174ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 82/100
15/15 [==============================] - 3s 177ms/step - loss: 0.0014 - val_loss: 0.0019
Epoch 83/100
15/15 [==============================] - 3s 174ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 84/100
15/15 [==============================] - 3s 180ms/step - loss: 0.0013 - val_loss: 0.0028
Epoch 85/100
15/15 [==============================] - 3s 174ms/step - loss: 0.0014 - val_loss: 0.0018
Epoch 86/100
15/15 [==============================] - 3s 178ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 87/100
15/15 [==============================] - 3s 174ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 88/100
15/15 [==============================] - 3s 178ms/step - loss: 0.0012 - val_loss: 0.0020
Epoch 89/100
15/15 [==============================] - 3s 178ms/step - loss: 0.0012 - val_loss: 0.0018
Epoch 90/100
15/15 [==============================] - 3s 177ms/step - loss: 0.0013 - val_loss: 

# Test

In [46]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

try:
    while(True): 
        ret, frame = cap.read()
        if ret:
            preds = landmarks_detector.get_landmarks(frame)
            try:
                left_eye = get_bound(preds[36:42])
                box = get_box(left_eye)
            except:
                pass
            img = warpBox(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), box, target_height=target_height, target_width=target_width) / 255.0
            img = np.expand_dims(np.expand_dims(img, 0), -1)
            [x, y] = model.predict(img)[0]
            pyautogui.moveTo(x * width, y * height, duration=0.15)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise GetOutOfLoop
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()